In [23]:
#!/usr/bin/python

'''
A program for network communication and control of the ISEG HV suppies
A. J. McCulloch, December 2018
'''

####################################################################################################
#Import modules
####################################################################################################

import requests #For API requests
import pandas as pd #For data frames
import numpy as np #For maths
import os #Operating system interfacing
import sys #System-specific parameters
import time #Time access and conversions
import getpass #Hides inputs when entering passwords
import websocket #Required for websocket communications
import json #Required for JSON file structure manipulation
from CFIBfunctions import * #Function definitions
from datetime import datetime, timezone, timedelta #For manipuation of time

####################################################################################################
#Define classes
####################################################################################################

#A class to house the iCS tasks
class iCStasks:
    
    #Status (meausrement)
    VMEAS = 'Status.voltageMeasure'
    ALIVE = 'Status.isAlive'
    STATE = 'Status.runningState'
    TMEAS = 'Status.temperature'
    #Control (set values)
    POWER = 'Control.on'
    VSET = 'Control.voltageSet'

    #Can be updated dynamically but not overwritten
    #Adapted from http://www.siafoo.net/snippet/108
    def __setattr__(self, attr, value):
        if hasattr(self, attr):
            raise ValueError('Attribute %s already has a value and so cannot be written to' % attr)

        self.__dict__[attr] = value
    
    #Initialise contact with the iCS2 unit, either via API or websocket
    @staticmethod
    def initialise(useAPI = False):
        
        #The session ID, loginflag and websocket are a global variables
        global sessionid
        global loginflag
        global ws
        
        #If initialse is called twice, have a flag to avoid logging in twice
        try:
            ws.connected
            try:
                loginflag
            except NameError:
                loginflag = True
        except NameError:
            loginflag = False
        
        if loginflag == False:     
            #Initialisation of iCS2 communication via web API
            if useAPI == True:
                #Returns API Key to be identified for session
                r = requests.get('http://'+ip+'/api/login/'+usr+'/'+passwd)
                try:
                    sessionid = r.json()['i']
                    print("Session successfully created")
                except KeyError:
                    sessionid = None
                    print("Session not created. Check password or iCS2 connection")

            #Initialisation of iCS2 communication over websocket
            else:
                executeWSrequest(generateWSrequest(stuct='login'))
                #The websocket needs to be purged to properly function (not sure why)
                #Close the websocket
                ws.close()
                #Reconnect (importantly not changing the session ID)
                ws.connect('ws://'+ip+':'+wsport)
        elif loginflag == True:
            print("Session previously established")
    
    #Close and reopen the websocket
    @staticmethod
    def resetsocket():        
        #The websocket is global
        global ws
        #Close the websocket
        ws.close()
        #Reconnect (importantly not changing the session ID)
        ws.connect('ws://'+ip+':'+wsport)
    
    #Terminate contact with the iCS2
    @staticmethod
    def terminate(useAPI = False):
        
        #The session ID is a global variable
        global sessionid
        
        #Termination of iCS2 communication via web API
        if useAPI == True:
            #Need to add the API turnoff command for turning off all channels
            #This is just a guess of the command! FIX THIS
            requests.get('http://'+ip+'/api/logout/')
        #Termination of iCS2 communication over websocket
        else:
            #Zero all set voltages
            zeroall()
            #Turn all channels off
            powerall()
            #Sever connection with the iCS2
            executeWSrequest(generateWSrequest(stuct='logout'))

#Define functions for communicating with ISEG iCS
class iCS2(object):
    
    #Set attributes
    def __init__(self, chid, address, limit, setpoint = 0, toset = 0, active = False, powered = False, measured = None):
        self.chid = chid #Channel ID
        ##self.address = address #Channel location [line, address, channel]
        self.address = address #Channel location {'l': line, 'a': address, 'c': channel}
        self.limit = limit #Channel limit
        self.setpoint = setpoint #Current channel setpoint
        self.toset = toset #Value to set when setting multiple electrodes
        self.active = active #Is the channel active
        self.powered = powered #Is the channel active
        self.measured = measured #Last measured voltage
        
    #Function for powering electrodes
    def power(self, turn_on = True):
        if turn_on == True:
            value = 1
        elif turn_on == False:
            value = 0
        #Generate and execute the websocket command to set the power status of the channel
        executeWSrequest(generateWSrequest(self, value, wstask = iCStasks.POWER))
            
    #Function for setting the voltage of a single electrode
    def setvoltage(self, voltage=0, units='V', useAPI = False):
        #Set the voltage via API
        if useAPI == True:
            interpretAPIresponse(exectureAPIrequest(generateAPIrequest(self, iCStasks.VSET, voltage, units)))
        #Set voltage over websocket
        else:
            executeWSrequest(generateWSrequest(self, voltage))
        if self.setpoint == voltage:
            print("Voltage of {} successfully set to {} {}".format(self.chid, voltage, units))
        else:
            print("Voltage of {} not updated".format(self.chid))
        
    #Function for measuring the voltage of a single electrode
    def measurevoltage(self, useAPI = False):
        #Measure the voltage using the API (slow)
        if useAPI == True:
            vmeas = interpretAPIresponse(exectureAPIrequest(generateAPIrequest(self,iCStasks.VMEAS)))
        #Get the voltage via websocket
        else:
            vmeas = executeWSrequest(generateWSrequest(self, wstask = iCStasks.VMEAS))
        
        self.measured = float(vmeas[0])
        if vmeas != None:
            print('The returned voltage was {} V [{}]'.format(vmeas[0],timestampconvert(time.time())))
        else:
            print('No voltage was returned')
            
    #Load the initial values for the experiment and power channels. Details stored in a file 
    #with name including 'filestring' located in directory
    @staticmethod
    def loadinitial(filestring, directory = None):
        #Read the contents of the most recent file containing the given string
        loadvoltages = readfile(getrecentfile(filestring, directory))
        #Split the list on new lines and then discard any elements beginning with #
        rawlist = [x for x in loadvoltages.replace(',','\n').splitlines() if not x.startswith('#')]
        #Get the active channels
        chans = [x for x in rawlist if x.startswith('e')]
        #Get the associated voltages and convert to floats
        values = [float(x) for x in rawlist if not x.startswith('e')]
        #Make a list of tuples
        cleanlist = list(zip(chans, values))
        
        for electrode in cleanlist:
            #Set the class object
            obj = str_to_class(electrode[0])
            voltage = electrode[1]
            #Set the active attribute to true
            obj.active = True
            #Set the toset attribute (if it is all within the limit)
            if voltage <= abs(obj.limit) and np.sign(voltage) == np.sign(obj.limit) or np.sign(voltage) == 0:     
                obj.toset = electrode[1]
            else:
                print('The set voltage {} for channel {} is outside the channel limit; it must be less than {} V'.format(voltage, obj.chid, obj.limit))
        
        #Generate a list of active electrodes
        activeelectrodes = list(filter(lambda x: x.active == True, eset))

        #Turn on active electrodes
        #If no active electrodes found, do nothing
        if len(activeelectrodes) == 0:
            print("No electrodes to power")
        #For a singe electrode, power it on
        elif len(activeelectrodes) == 1:
            activeelectrodes[0].power()
        #For multiple channels, make a single task and then execute
        else:
            #Initialise the multitask list
            multitask = []
            #Loop is used rather than list comprehension to update the power attribute in the same loop
            for e in activeelectrodes:
                #Append tasks that are requested
                multitask.append(generateWSrequest(e, 1, wstask = iCStasks.POWER))
                #Update the attribute. Not ideal as the request has not even been submitted, but it is not part of executeWSrequest at this stage
                e.power = True
            #Execute the request
            executeWSrequest(multitask)
        
        return activeelectrodes
    
    #Set any voltages which have a toset attribute different to their setpoint attribute
    @staticmethod
    def multisetvoltage():
        #Generate a list of active electrodes with values to change
        electrodestoset = list(filter(lambda x: x.setpoint != x.toset, activeelectrodes))
        #Set the voltages:
        #Case 1: No voltage to update
        if len(electrodestoset) == 0:
            print("No electrode voltages to change")
        #Case 2: For a singe voltage
        elif len(electrodestoset) == 1:
            electrodestoset[0].setvoltage(electrodestoset[0].toset)
        #Case 3: For multiple voltages, generate a single task and then execute
        else:
            #Generate a list of requests
            reqlist = [generateWSrequest(x,x.toset) for x in electrodestoset]
            #Execute the request
            executeWSrequest(reqlist)
    
####################################################################################################
#Define API related functions
####################################################################################################

"""
Functions that are used interfacing with the iCS2 with the web API
"""
#Generate a request to interface with the iCS. Inputs are electode ID (str) and task (str)
#Designed to be used with iCS2 attributed objects (obj, iCStasks, voltage, unit)
def generateAPIrequest(eid, task, voltage=0, unit='V'):
    #Verify channel ID is actual channel
    if eid.address in chlist:
        #Verify the task is a registered task
        #NOTE: this is a tasty little number
        if task in [getattr(iCStasks,y) for y in [x for x in iCStasks.__dict__.keys() if not x.startswith('_')]]:
            
            #Voltage measure
            if task == 'Status.voltageMeasure':
                #Generate the appropriate address string, must be of the form /*/*/*/
                #tloc = ''.join(['/'+str(x) for x in eid.address])+'/'
                tloc = ''.join(['/'+str(x) for x in [eid.address[i] for i in addparam]])+'/'
                #Generate the request
                req = apiget+sessionid+tloc+task
                return req
            
            #Voltage set
            elif task == 'Control.voltageSet':
                #Generate the appropriate address string, must be of the form /*/*/*/
                ##tloc = ''.join(['/'+str(x) for x in eid.address])+'/'
                tloc = ''.join(['/'+str(x) for x in [eid.address[i] for i in addparam]])+'/'
                #Verify voltage unit
                if unit in ('V', 'kV'):
                    #Scale kV voltages to V
                    if unit == 'kV':
                        voltage *= 1000
                    #Verify voltage is less than channel limit
                    if abs(voltage) <= abs(eid.limit) and np.sign(voltage) == np.sign(eid.limit) \
                    or np.sign(voltage) == 0:
                        #Generate voltage string
                        setval = ''.join('/'+str(voltage)+'/'+unit)
                        #Generate the request
                        req = apiset+sessionid+tloc+task+setval
                        #Update the setpoint attribute    
                        eid.setpoint = voltage
                    else:
                        print('The set voltage {} for channel {} is outside the channel limit; it must be less \
                        than {} V'.format(voltage, eid.chid, eid.chid))
                        req = None
                else:
                    print('Invalid unit for voltage of channel {}; use either V or kV'.format(eid.chid))
                    req = None
                return req
        
        #Inform if task is not registered task
        else:
            print('The specified task {} does not exist. Registered tasks are stored in the iCStasks class'.format(task))
    #Inform of incorrect IDs        
    else:
        print('The channel ID {} does not exist. The channel ID must be registered; for a \
        list of registered IDs, execute print(elabels)'.format(eid))
        
#Execute a request and return the response
def exectureAPIrequest(request):
    req = requests.get(request)
    return req

#Extract information from a response
def interpretAPIresponse(response):
    #Catch content in response
    try:
        #Convert response to useful list by extracting content of response
        content = response.json()[0]['c']
        if len(content) == 0:
            print('No content is returned in response')
            #Return nothing
            ds = None
        elif len(content) == 1:
            #Return a dictionary with the embedded data
            ds = content[0]['d']
        else:
            #Return a list of dictionaries of the embedded data
            ds=[content[x]['d'] for x in range(len(content))]

        #In the case of of voltage measurement, print the voltage (timestamped)
        if ds['i'] == iCStasks.VMEAS:
            """
            20190122
            This is not working, the offest has issues, resulting in incorrect times
            It is in the bin for the moment.
            
            #Check to see if the time offset has been calculated
            try:
                tfix
            except NameError:
                tfix = timefix()
            else:
                pass     

            #print('The returned voltage was {} V [{}]'.format(ds['v'],timestampconvert(float(ds['t']),tfix)))
            """
            #Time is using (local) system time
            print('The returned voltage was {} V [{}]'.format(ds['v'],timestampconvert(time.time())))
    
    #If not content in response, look for trigger (information given in docs)
    #http://192.168.68.237//doc/iCSservice/iCSapiWebsocket_Docu.html
    except KeyError:
        content = response.json()[0]['trigger']
        if content == 'false':
            print('Server can not handle request or wrong input information given')
        elif content == 'true':
            print('Server has (suscessfully) processed the request')
        elif content == 'denied':
            print('Server denies request because of access restrictions')
        elif 'noData':
            print('No data available to this request (on getUpdate command)')
        else:
            pass
        ds = None
    return ds

####################################################################################################
#Define websocket related functions
####################################################################################################
        
#Function to generate the websocket tasks. Mainly geared for setting electrode values
#Inputs are electrode address (e.address), the set voltage, the task to complete and the structure type (usally request but can be login/logout)
#The plan is for a task to be called and if the input setall is True, then all channels are returned
#For toggling power via iCStasks.POWER, the voltage variable is a boolean for on/off
def generateWSrequest(eid = None, voltage = 0, units = 'V', wstask = iCStasks.VSET, stuct = 'request', setall = False):
        
    #The session ID is a global variable
    global sessionid
    
    #Session mode ("websocket" or "xhr"), see iCS2 docs
    r = 'websocket'
    #Structure type
    t1 = stuct
    
    #Generate login task parameters, must be {"i": "", "t": "login", "c": {"l": "user", "p": "password", "t": ""}, "r": "websocket"} 
    if t1 == 'login':
        #Session ID
        i = ''
        #Username
        l = usr
        #Password
        try:
            p = passwd
        except NameError:
            p = getpass.getpass("Password for iCS2 module:")
        t2 = ''
        c1 = {'l': l, 'p': p, 't': t2}    
    
    #Generate logout task parameters, must be {"i": session ID, "t": "logout", "c": {}, "r": "websocket"}
    elif t1 == 'logout':
        i = sessionid
        c1 = {}
    
    #Generate requst task
    elif t1 == 'request':
        #Session ID
        i = sessionid
        
        #Generate task for setting the voltage, must be {"i": session ID, "t": "request", "c": [{"c": "setItem","p": {"p": {"l": "0","a": "2", "c": "1"},"i": "Control.voltageSet","v": "0","u": "V"}}], "r": "websocket"}
        if wstask == iCStasks.VSET:
            
            #Verify unit of voltage is correct
            if units in ('V', 'kV'):
                #To set all voltages to the same value (really only useful for zeroing)
                if setall == True:
                    c2 = 'setItem'
                    [line, add, chan] = ['*'] * 3
                    item = wstask
                    v = voltage
                    u = ''
                    p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': item, 'v': v, 'u': u}
                    #Put the instructions together
                    c1 = [{'c': c2, 'p': p}]
                    
                #Set individual channel voltage
                elif setall == False:
                    #Verify voltage is less than channel limit
                    if abs(voltage) <= abs(eid.limit) and np.sign(voltage) == np.sign(eid.limit) \
                    or np.sign(voltage) == 0:                    
                        c2 = 'setItem'
                        item = wstask
                        v = voltage
                        u = units
                        p = {'p': eid.address, 'i': item, 'v': v, 'u': u}
                        #Put the instructions together
                        c1 = [{'c': c2, 'p': p}]
                        #Update the setpoint attribute    
                        eid.setpoint = voltage
                    else:
                        print('The set voltage {} for channel {} is outside the channel limit; it must be less \
                        than {} V'.format(voltage, eid.chid, eid.limit))
                        c1 = None
            else:
                print('Invalid unit for voltage of channel {}; use either V or kV'.format(eid.chid))
                c1 = None     
        
        #Generate task for measuring the voltage
        elif wstask == iCStasks.VMEAS:
            #Measure all channel voltages
            if setall == True:
                c2 = 'getItem'
                [line, add, chan] = ['*'] * 3
                item = wstask
                v = ''
                u = ''
                p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': item, 'v': v, 'u': u}
                #Put the instructions together
                c1 = [{'c': c2, 'p': p}]
            
            #Measure the voltage of a single channel
            elif setall == False:
                c2 = 'getItem'
                item = wstask
                v = ''
                u = ''
                p = {'p': eid.address, 'i': item, 'v': v, 'u': u}
                #Put the instructions together
                c1 = [{'c': c2, 'p': p}]
        
        #Generate task for toggling power, must be {"i": sessionid, "t": "request", "c": [{"c": "setItem", "p": {"p": {"l": "0", "a": "0", "c": "0"}, "i": "Control.on", "v": "1", "u": ""}}]}
        elif wstask == iCStasks.POWER:
            #The set value must be 1 or 0 (on or off)
            if voltage in (0,1):
                #Generate task to power all channels
                if setall == True:
                    c2 = 'setItem'
                    [line, add, chan] = ['*'] * 3
                    item = wstask
                    v = voltage
                    u = ''
                    p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': item, 'v': v, 'u': u}
                    #Put the instructions together
                    c1 = [{'c': c2, 'p': p}]
                
                #Generate task for single channel
                elif setall == False:
                    c2 = 'setItem'
                    item = wstask
                    v = voltage
                    u = ''
                    p = {'p': eid.address, 'i': item, 'v': v, 'u': u}
                    #Put the instructions together
                    c1 = [{'c': c2, 'p': p}]
                    if voltage == 1:
                        #Update the power attribute    
                        eid.power = True
                    elif voltage == 0:
                        #Update the power attribute    
                        eid.power = False
            #If the set value is not 1 or 0, return an error                    
            else:
                print("The control value {} is invalid, it must be either 0 or 1".format(voltage))
                c1 = None
        
        #Generate task for measuring the status of the channel
        elif wstask == iCStasks.STATE:
            #Generate task for return of all channels
            if setall == True:
                c2 = 'getItem'
                [line, add, chan] = ['*'] * 3
                item = wstask
                v = ''
                u = ''
                p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': item, 'v': v, 'u': u}
                #Put the instructions together
                c1 = [{'c': c2, 'p': p}]
                
            #Generate task for single channel
            elif setall == False:
                c2 = 'getItem'
                [line, add, chan] = ['*'] * 3
                item = wstask
                v = ''
                u = ''
                p = {'p': eid.address, 'i': wstask, 'v': v, 'u': u}
                #Put the instructions together
                c1 = [{'c': c2, 'p': p}]
        
        """
        #Generate task for toggling power of all channels
        elif wstask == 'POWER_all':
            if voltage in (0,1):                    
                c2 = 'setItem'
                [line, add, chan] = ['*'] * 3
                item = wstask
                v = voltage
                u = ''
                p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': iCStasks.POWER, 'v': v, 'u': u}
                #Put the instructions together
                c1 = [{'c': c2, 'p': p}]
                                
            else:
                print("The control value {} is invalid, it must be either 0 or 1".format(voltage))
                c1 = None
                
        #Generate task for zeroing all set point voltages
        elif wstask == 'SET_all':                   
            c2 = 'setItem'
            [line, add, chan] = ['*'] * 3
            item = wstask
            v = voltage
            u = ''
            p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': iCStasks.VSET, 'v': v, 'u': u}
            #Put the instructions together
            c1 = [{'c': c2, 'p': p}]
            
        #Generate task for measuring all voltages
        elif wstask == 'VMEAS_all':
            c2 = 'getItem'
            [line, add, chan] = ['*'] * 3
            item = wstask
            v = ''
            u = ''
            p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': iCStasks.VMEAS, 'v': v, 'u': u}
            #Put the instructions together
            c1 = [{'c': c2, 'p': p}]
        """

    #The generated task (see iCS2 docs for unclear discussion of format)
    gentask = {'i': i, 't': t1, 'c': c1, 'r': r}
    #Return the generated task
    return gentask        

#A function to execute websocket tasks (usually generated with "wstaskgen")
def executeWSrequest(task, responselimit = 5):
    
    #The session ID, loginflag and websocket are global variables
    global sessionid
    global loginflag
    global ws
    
    #Verify websocket has been created
    try:
        ws
    except NameError:
        #If socket has not been created, create it
        ws = websocket.create_connection('ws://'+ip+':'+wsport)
        
    #Verify the websocket is connected (catch created but closed websockets)
    if ws.connected == False:
        #Open the websocket
        ws = websocket.create_connection('ws://'+ip+':'+wsport)
    
    #If session ID exists, one can use the websocket connection
    try:
        sessionid
    #If session ID is not defined, use websocket to initiate session
    except NameError:
        #If executing a login command, don't establiash a connection
        if task['t'] == 'login':
            pass
        #Otherwise, establish a connection
        else:
            #Open websocket connection
            try:
                ws.send(json.dumps(wstaskgen(stuct = 'login')))
            except WebSocketConnectionClosedException:
                print("Websocket is closed, check connection to iCS2")

            #Get the session ID
            try:
                sessionid = json.loads(ws.recv())['i']
                print("No existing session found. Websocket communication established")
            except KeyError:
                sessionid = None
                print("No existing session found however connection failed. Check password or iCS2 connection")

            #Set a flag to we don't make two connections
            loginflag = True
    
    #Seperate individual tasks from mulitple tasks; multiple tasks will be given as lists
    if type(task) == dict:
        
        #Prepare data packet
        packet = json.dumps(task)

        #For login task
        if task['t'] == 'login':
            if loginflag == True:
                #If the flag is true, login has occured
                pass
            else:
                #Send data packet
                try:
                    ws.send(packet)
                except WebSocketConnectionClosedException:
                    print("Websocket is closed, check connection to iCS2")

                try:
                    sessionid = json.loads(ws.recv())['i']
                    print("Websocket communication sucessfully established")
                    loginflag = True
                except (KeyError, TypeError):
                    print("Websocket not established, check password")
            response = None

        #For logout task
        elif task['t'] == 'logout':
            #Send data packet
            try:
                ws.send(packet)
            except WebSocketConnectionClosedException:
                print("Websocket is closed, check connection to iCS2")

            print("Websocket server logout completed")
            ws.close()
            print("Websocket closed")
            loginflag = False
            response = None

        #For request task
        elif task['t'] == 'request':
            #Catch invalid tasks (generated from incorret set limits)
            if task['c'] != None:
                #Setting a value 
                if task['c'][0]['c'] == 'setItem':
                    #Send data packet
                    try:
                        ws.send(packet)
                    except WebSocketConnectionClosedException:
                        print("Websocket is closed, check connection to iCS2")

                    response = None
                #Requesting a value
                elif task['c'][0]['c'] == 'getItem':
                    #Send data packet
                    try:
                        ws.send(packet)
                    except WebSocketConnectionClosedException:
                        print("Websocket is closed, check connection to iCS2")

                    reclimit = 0
                    while reclimit < responselimit:
                        received = json.loads(ws.recv())            
                        response = interpretWSresponse(received, task)
                        #Nothing useful was returned
                        if response == None:
                            reclimit += 1
                            if reclimit == responselimit - 1:
                                print('No response for iCS2, check connection')
                                reclimit = responselimit
                        else:
                            #If 'trigger' is returned, the server cannot handle the request
                            try:                
                                received[0]['trigger']
                                reclimit += 1
                                if reclimit == responselimit - 1:
                                    print('Invalid request, check channel address')
                                    reclimit = responselimit
                            #Something useful was returned
                            except (KeyError, TypeError):
                                reclimit = responselimit
            else:
                response = None
                pass
    
    #When a collection of instructions is given it must be a list
    elif type(task) == list:
        #There is much less oversight here. Perhaps in the future some proofing can be implemented
        
        """    
        #Here is some code to determine if all tasks are identical - likely given how the code is structured
        #It would be useful to update the electrode attributes here rather than in task generation, but that is for the future
        
        if all_same([x['c'][0]['p']['i'] for x in task]):
            thetask = task[0]['c'][0]['p']['i']
                if thetask == iCStasks.VSET
                #PUT CODE TO UPDATE SETPOINT
        """
        
        #The commands need to be extracted from the tasks and reassembled to send
        #Setp 1: get the instructions
        instructions = []
        instructions.extend(flattenlist([tsk['c'] for tsk in task]))
        #Step 2: make a new packet with the full instructions. The task[0] element is used to get the vitals (session id etc.)
        multipacket = task[0]
        multipacket['c'] = instructions
        #Prepare the data packet
        packet = json.dumps(multipacket)
        
        #Send the data packets
        try:
            ws.send(packet)
        #Catch closed connections
        except WebSocketConnectionClosedException:
            print("Websocket is closed, check connection to iCS2")
            
        #Nothing to return
        response = None
    
    #Return the response    
    return response      

#Interperet websocket response for a given task (function called from executews)
def interpretWSresponse(response, task):
    responsecontent = response[0]['c']
    #Single response
    if len(responsecontent) == 1:
    #Interpret the response
        try:
            #Get the data packet from the respose
            recdatadict = responsecontent[0]['d']
        except (KeyError, TypeError):
            print("No data packet returned from iCS2 (check channel address)")
            recdatadict = None

        #Process the content of the data packet
        if recdatadict != None:
            #Extract the useful information from returned data
            try:
                datakeys = ['i','p','v','u','t']
                #Get the data
                recdata = [recdatadict.get(key) for key in datakeys]
            except KeyError:
                print("Unexpected data returned from iCS2")
                datakeys, recdata = None, None

            #Check the data is what was requested
            if recdata != None and recdata[:2] == [task['c'][0]['p'].get(key) for key in ['i','p']]:
                #Store the useful data
                measdata = recdata[2:]
            else:
                measdata = None
                print('Returned data different from requested information')
        else:
            measdata = None
            
    #Response with multiple packets
    else:
        #Check if the tasks are the same. If not, it is likely a bad response.
        #The tasks listed in the response
        resptasks = [x['d']['i'] for x in responsecontent]
        #The tasks should be the same
        if all_same(resptasks):
            thetask = resptasks[0]
            
            #Returns a 0 if voltage ramping complete, returns a 1 if ramping
            if thetask == iCStasks.STATE:
                #initialise data string
                recdata = []
                #Data to extract. For status, only care about the address and the value
                datakeys = ['p','v']
                #Make a list of elements we care about
                for d in response[0]['c']:
                    recdata.append([d['d'].get(key) for key in datakeys])
                #Keep only the top level data - each line of the supply
                recdata = [x for x in recdata if x[0]['c'] == '' and x[0]['a'] != '1000']
                
                #Interperet the data
                statuses = list(set([x[1] for x in recdata]))
                #The whole point of checking the status is to verify ramping has finished
                if 'info' in statuses:
                    #print("Voltage(s) ramping")
                    measdata = 1
                else:
                    #print("Voltage(s) stable)
                    measdata = 0
            #String of the same measurement returned, but not what we want
            else:
                measdata = None
        #Don't return anything
        else:
            measdata = None
            print('Returned data different from requested information')

    return measdata

####################################################################################################
#Define functions
####################################################################################################
#Take a string and convert it to a class object
def str_to_class(str):
    return getattr(sys.modules[__name__], str)

#Switch the power to all channels. By default, this will turn off all channels
def powerall(turnon = False):
    
    #eset is global
    global eset
    
    toupdate = eset
    #Allow all channels to be powered on
    if turnon == True:
        value = 1
        #Update the powered attribute
        for e in toupdate:
            e.powered = True
    #All channels to be powered off (more useful)
    elif turnon == False:
        value = 0
        #Update the powered attribute
        for e in toupdate:
            e.powered = False
    #Generate and execture the websocket request to power the channels
    executeWSrequest(generateWSrequest(voltage = value, wstask = iCStasks.POWER, setall = True))

#Set all voltages to zero
def zeroall():
    #eset is global
    global eset
    
    toupdate = eset
    #Generate and execture the websocket request to power the channels
    executeWSrequest(generateWSrequest(voltage = 0, wstask = iCStasks.VSET, setall = True))
    #Update the toset and setpoint attributes
    for e in toupdate:
        e.toset = 0
        e.setpoint = 0

#Determine if voltages are stable (if any are ramping). Returns True if stable
def voltagestable():
    if executeWSrequest(generateWSrequest(wstask = iCStasks.STATE, setall = True)) == 0:
        return True
    else:
        return False
        
#Return a dataframe of all measured voltages
def measureall(task = iCStasks.VMEAS, useAPI = False):
    #Verify the task is a registered task
    #NOTE: this is a tasty little number
    if task in [getattr(iCStasks,y) for y in [x for x in iCStasks.__dict__.keys() if not x.startswith('_')]]:
        if useAPI == True:
            #Use API to pull data, returns a response
            rquest = requests.get(apiget+sessionid+'/*/*/*/'+task)
            #Convert response to useful list by extracting content of response
            content = rquest.json()[0]['c']
            
        else:
            request = generateWSrequest(wstask = iCStasks.VMEAS, setall = True)
            try:
                ws.send(json.dumps(request))
            except WebSocketConnectionClosedException:
                print("Websocket is closed, check connection to iCS2")
                
            content = json.loads(ws.recv())[0]['c']
        #The goal is to load the data into a dataframe
        #Step 1: Clean/prepare the data
        #Return a list of dictionaries of the embedded data
        ds=[content[x]['d'] for x in range(len(content))]
        
        #Convert the addresses to electrode label (iCS.chid)
        for i in range(len(ds)): 
            try:
                #Find the electrode
                electrode = chaddressdict[str(ds[i]['p'])]
                #Change the identifier from address to the label
                ds[i]['p'] = str_to_class(electrode).chid
                #Update the last measured value
                str_to_class(electrode).measured = float(ds[i]['v'])
            #There will be extra channels returned, give these a None definition
            except KeyError:
                ds[i]['p'] = None
          
        #Step 2: Put data in a dataframe, then clean it
        df = pd.DataFrame(ds)
        #Rename the dataframe columns
        df.columns = ['Task','Electrode','Time','Unit','Voltage']
        #Reorder the columns
        df = df[['Electrode','Task','Voltage','Unit','Time']]
        #Set 'None' addresses to NaN
        df.Electrode.fillna(value=pd.np.nan, inplace=True)
        #Drop the rows with NaN
        df = df.dropna(subset=['Electrode'])
        #Convert the time
        #df['Time'] = df['Time'].apply(lambda t:timestampconvert(float(t)))
        #Drop the task and time (at least until the time server issue is fixed)
        df = df.drop(columns=['Task','Time'])

    else:
        print('The specified task {} does not exist. Registered tasks are stored in the iCStasks class'.format(task))
        df = None
    
    return df

####################################################################################################
####################################################################################################
#Code starts here
####################################################################################################        
####################################################################################################

####################################################################################################
#Program initialisation
####################################################################################################

#The session must be open for communication with the iCS2
iCStasks.initialise()

#Initialise the electrode objects and attributes
try:
    e1.chid
except NameError:
    #Create an empty list to store the electrode objects
    eset = []
    #Set attributes for each electrode from the elabels list
    for e in elabels:
        #Create each of the electrode objects
        vars()[e] = iCS2(*[x[elabels.index(e)] for x in [chids,chlist,limits]])
        #Append each to the electrode set list
        eset.append(str_to_class(e))

####################################################################################################
#Program/Sequence code
####################################################################################################

Session previously established


In [ ]:
#Measure voltages of interest

In [27]:
#Initialisation is with a list of tuples, the electrode and the desired voltage
#If the electrode is not listed, the electrode will not be powered (i.e. the voltage will float!)
activeelectrodes = iCS2.loadinitial('Initialisation-voltages')

#Ppower on the voltages as loaded from the initialisation file
#iCS2.multisetvoltage()

Last file update occured on 2019-02-12 16:34:24 (AUS Eastern Daylight Time)


In [39]:
measureall()
pd.DataFrame([vars(e) for e in eset])  

,active,address,chid,limit,measured,powered,setpoint,toset
0,False,"{'l': '0', 'a': '0', 'c': '0'}",+1kV1,1000,0.019,False,0,0
1,False,"{'l': '0', 'a': '0', 'c': '1'}",+1kV2,1000,0.016,False,0,0
2,False,"{'l': '0', 'a': '0', 'c': '2'}",+1kV3,1000,0.005,False,0,0
3,False,"{'l': '0', 'a': '0', 'c': '3'}",+1kV4,1000,0.001,False,0,0
4,False,"{'l': '0', 'a': '0', 'c': '4'}",-1kV1,-1000,-0.047,False,0,0
5,False,"{'l': '0', 'a': '0', 'c': '5'}",-1kV2,-1000,-0.017,False,0,0
6,False,"{'l': '0', 'a': '0', 'c': '6'}",-1kV3,-1000,-0.012,False,0,0
7,False,"{'l': '0', 'a': '0', 'c': '7'}",-1kV4,-1000,-0.359,False,0,0
8,False,"{'l': '0', 'a': '1', 'c': '0'}",+20kV1,20000,-0.108,False,0,0
9,False,"{'l': '0', 'a': '1', 'c': '1'}",+20kV2,20000,-0.050,False,0,0


# Example program code

In [2]:
#Set single voltage
e1.setvoltage(0)

Voltage of +1kV1 successfully set to 0 V


In [13]:
#Measure single voltage
e1.measurevoltage()

The returned voltage was 0.023 V [2019-02-13 17:45:55 (AUS Eastern Daylight Time)]


In [16]:
#Measure all voltages
df = measureall()
df

,Electrode,Voltage,Unit
0,+1kV1,0.025,V
1,+1kV2,0.023,V
2,+1kV3,0.013,V
3,+1kV4,0.008,V
4,-1kV1,-0.047,V
5,-1kV2,-0.025,V
6,-1kV3,-0.021,V
7,-1kV4,-0.359,V
8,+20kV1,-0.152,V
9,+20kV2,-0.055,V


In [ ]:
#Test if voltages are stable (ramping)
voltagestable()

In [55]:
#Zero all set points
zeroall()

In [56]:
#Turn all channels off
powerall()

In [40]:
#Logout (zeros and powers off all channels, disconnects the from the iCS2 and closes the websocket)
iCStasks.terminate()

Websocket server logout completed
Websocket closed
